# Bolsa de palabras en paralelo

Mateo De La Roche 190748 

Sebastian Murillo 

Frida Kaori Castañeda

In [3]:
#librerias utilizadas

import pandas as pd
import numpy as np
import re
import nltk
import matplotlib.pyplot as plt

## Corpus etiquetado

In [25]:
etiquetas = ["shakespeare_the_merchant_of_venice", 
             "shakespeare_romeo_juliet", 
             "shakespeare_hamlet", 
             "dickens_a_christmas_carol", 
             "dickens_oliver_twist",
             "dickens_a_tale_of_two_cities"]
corpus = []

for etiqueta in etiquetas:
  archivo = open(f"data/{etiqueta}.txt", "r")
  corpus.append(archivo.read())
  archivo.close()

corpus = np.array(corpus)
df_corpus = pd.DataFrame({"documento": corpus, 
                          "categoria": etiquetas})
df_corpus

,documento,categoria
0,"project,gutenberg,ebook,merchant,venice,willia...",shakespeare_the_merchant_of_venice
1,"project,gutenberg,ebook,romeo,juliet,william,s...",shakespeare_romeo_juliet
2,"project,gutenberg,ebook,hamlet,william,shakesp...",shakespeare_hamlet
3,"project,gutenberg,ebook,christmas,carol,charle...",dickens_a_christmas_carol
4,"project,gutenberg,ebook,oliver,twist,charles,d...",dickens_oliver_twist
5,"project,gutenberg,ebook,tale,two,city,charles,...",dickens_a_tale_of_two_cities


En caso de haber cambiado los libros se debera cambiar la variable ```string libros[libros_size]``` en los codigos de c++.

La variable debera ser declarada con la salida de la siguiente celda.

In [26]:
#empty string
out = "{"
for etiqueta in etiquetas:
    out += '"' + etiqueta + '",'
out = out[:-1]
out += "}"
print(out)

{"shakespeare_the_merchant_of_venice","shakespeare_romeo_juliet","shakespeare_hamlet","dickens_a_christmas_carol","dickens_oliver_twist","dickens_a_tale_of_two_cities"}


## Modelo de Bolsa de Palabras

In [27]:
from sklearn.feature_extraction.text import CountVectorizer
# bolsa de palabras en matriz dispersa
count_vectorizer = CountVectorizer(min_df=0.0, max_df=1.0)
matriz_conteo = count_vectorizer.fit_transform(corpus)
# ver valores diferentes de cero en la matriz dispersa
print(matriz_conteo)

  (0, 10315)	90
  (0, 6108)	99
  (0, 4317)	20
  (0, 8386)	20
  (0, 14294)	45
  (0, 14871)	4
  (0, 11781)	5
  (0, 1652)	8
  (0, 8214)	17
  (0, 11189)	18
  (0, 59)	2
  (0, 7745)	36
  (0, 13227)	12
  (0, 567)	5
  (0, 7486)	1
  (0, 1783)	11
  (0, 9390)	5
  (0, 1065)	1
  (0, 4611)	1
  (0, 6611)	1
  (0, 14849)	1
  (0, 183)	1
  (0, 13475)	8
  (0, 85)	7
  (0, 1710)	6
  :	:
  (5, 8286)	1
  (5, 10631)	1
  (5, 4475)	1
  (5, 5110)	1
  (5, 7418)	1
  (5, 2381)	1
  (5, 8105)	1
  (5, 829)	1
  (5, 2223)	1
  (5, 3206)	1
  (5, 3277)	1
  (5, 1047)	1
  (5, 4837)	1
  (5, 6348)	1
  (5, 244)	2
  (5, 11380)	1
  (5, 9754)	1
  (5, 14795)	1
  (5, 13303)	1
  (5, 13858)	1
  (5, 8390)	1
  (5, 11826)	1
  (5, 9584)	1
  (5, 4822)	1
  (5, 696)	1


la siguiente matriz y dataframe fueron los que se obtuvieron de las implementaciones seriales y paralelas en c++.

In [28]:
# ver la representación densa
matriz_conteo = matriz_conteo.toarray()
matriz_conteo

array([[1, 0, 0, ..., 0, 0, 0],
       [3, 0, 0, ..., 1, 0, 2],
       [3, 0, 1, ..., 2, 1, 0],
       [1, 1, 0, ..., 1, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 1, 0, 0]], dtype=int64)

In [29]:
# obten todas las palabras únicas del corpus
vocabulario = count_vectorizer.get_feature_names_out()
# muestra los vectores de características del documento
pd.DataFrame(matriz_conteo, columns=vocabulario, index=etiquetas)

,000,00033,04,08,0em,10,100,101,102,103,...,yourn,youth,youthful,youthfulness,zeal,zealous,zenith,zip,zone,zounds
shakespeare_the_merchant_of_venice,1,0,0,0,2,0,0,0,0,0,...,0,8,1,0,1,0,0,0,0,0
shakespeare_romeo_juliet,3,0,0,1,0,2,1,0,0,0,...,0,6,3,0,0,0,0,1,0,2
shakespeare_hamlet,3,0,1,0,0,2,3,0,0,0,...,0,14,0,0,0,0,0,2,1,0
dickens_a_christmas_carol,1,1,0,0,0,2,2,1,1,1,...,0,1,0,0,1,0,0,1,0,0
dickens_oliver_twist,1,0,0,0,2,0,0,0,0,0,...,0,9,6,0,0,2,1,0,0,0
dickens_a_tale_of_two_cities,1,0,0,0,0,2,0,0,0,0,...,1,10,3,1,0,2,0,1,0,0


## Vocabulario (número de columnas de la matriz)

En caso de haber cambiado los textos igual se debera volver a guardar el vocabulario corriendo la celda siguiente.


In [30]:
print(len(vocabulario), vocabulario)
np.savetxt('data/vocabulario.txt', vocabulario, fmt='%s')

15164 ['000' '00033' '04' ... 'zip' 'zone' 'zounds']


## Comandos para correr el codigo en c++

In [36]:
print(f".\\Parallel-bag-of-words\\bag-of-serial-words.exe {len(vocabulario)}")
print(f"mpiexec -n {len(etiquetas)} Parallel-bag-of-words\\bag-of-parallel-words.exe {len(vocabulario)}")


.\Parallel-bag-of-words\bag-of-serial-words.exe 15164
mpiexec -n 6 Parallel-bag-of-words\bag-of-parallel-words.exe 15164


## Resultados de la implementación en C++

### Versión serial

In [13]:
res_serial = pd.read_csv("res/bow-s-df.csv", encoding="latin1")
res_serial

,Libro,000,00033,04,08,0em,10,100,101,102,...,youth,youthful,youthfulness,zeal,zealous,zenith,zip,zone,zounds,Unnamed: 15165
0,shakespeare_the_merchant_of_venice,1,0,0,0,2,0,0,0,0,...,8,1,0,1,0,0,0,0,0,NaN
1,shakespeare_romeo_juliet,3,0,0,1,0,2,1,0,0,...,6,3,0,0,0,0,1,0,2,NaN
2,shakespeare_hamlet,3,0,1,0,0,2,3,0,0,...,14,0,0,0,0,0,2,1,0,NaN
3,dickens_a_christmas_carol,1,1,0,0,0,2,2,1,1,...,1,0,0,1,0,0,1,0,0,NaN
4,dickens_oliver_twist,1,0,0,0,2,0,0,0,0,...,9,6,0,0,2,1,0,0,0,NaN
5,dickens_a_tale_of_two_cities,1,0,0,0,0,2,0,0,0,...,10,3,1,0,2,0,1,0,0,NaN


### Versión paralela

In [14]:
res_serial = pd.read_csv("res/bow-p-df.csv", encoding="latin1")
res_serial

,Libro,000,00033,04,08,0em,10,100,101,102,...,youth,youthful,youthfulness,zeal,zealous,zenith,zip,zone,zounds,Unnamed: 15165
0,shakespeare_the_merchant_of_venice,1,0,0,0,2,0,0,0,0,...,8,1,0,1,0,0,0,0,0,NaN
1,shakespeare_romeo_juliet,3,0,0,1,0,2,1,0,0,...,6,3,0,0,0,0,1,0,2,NaN
2,shakespeare_hamlet,3,0,1,0,0,2,3,0,0,...,14,0,0,0,0,0,2,1,0,NaN
3,dickens_a_christmas_carol,1,1,0,0,0,2,2,1,1,...,1,0,0,1,0,0,1,0,0,NaN
4,dickens_oliver_twist,1,0,0,0,2,0,0,0,0,...,9,6,0,0,2,1,0,0,0,NaN
5,dickens_a_tale_of_two_cities,1,0,0,0,0,2,0,0,0,...,10,3,1,0,2,0,1,0,0,NaN


## Speedup

In [41]:
#cada vez que se corra el código de c++ se agregara una nueva fila al archivo
#esto se hace para que sea más presiso el tiempo de ejecución
df_tiempo = pd.read_csv("res/tiempos.csv", names=["vocabulario","modo", "tiempo"])
df_tiempo

,vocabulario,modo,tiempo
0,15164,serial,264695.0
1,15164,serial,230569.0
2,15164,parallel,95744.8
3,15164,parallel,92729.3


In [47]:
#Speedup obtenido 
df_s = df_tiempo[df_tiempo.modo == "serial"].groupby("vocabulario").mean().reset_index()
df_p = df_tiempo[df_tiempo.modo == "parallel"].groupby("vocabulario").mean().reset_index()

#df_speedup donde primera columna es el vocabulario y la segunda es el speedup 
df_speedup = pd.DataFrame({"vocabulario": df_s.vocabulario,
                            "speedup": df_s.tiempo / df_p.tiempo})
df_speedup  


C:\Users\mateo\AppData\Local\Temp\ipykernel_21384\3914780134.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_s = df_tiempo[df_tiempo.modo == "serial"].groupby("vocabulario").mean().reset_index()
C:\Users\mateo\AppData\Local\Temp\ipykernel_21384\3914780134.py:3: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_p = df_tiempo[df_tiempo.modo == "parallel"].groupby("vocabulario").mean().reset_index()


,vocabulario,speedup
0,15164,2.627756
